In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Load libraries

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

sns.set_style('whitegrid')

In [ ]:
# load train and test data
train_data = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test_data = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')
train_data.head()

In [ ]:
# Take a look at shapes
print('Train data shape:' , train_data.shape)
print('Test data shape:' , test_data.shape)

In [ ]:
train_data.info()

We can see that the training dataset has no missing values

In [ ]:
test_data.info()

The test datset also has no missing values

In [ ]:
# Extract continuous and categorical variables
cont_cols = [col for col in train_data.columns if train_data[col].dtype == 'float64']
cat_cols = [col for col in train_data.columns if train_data[col].dtype == 'object']
print(cont_cols)
print(cat_cols)

In [ ]:
train_data[cont_cols].describe()

It appears that all of the continous variables have already been scaled, so we won't need to do this ourselves.

In [ ]:
num_cats = []

for col in cat_cols:
    cats = train_data[col].nunique()
    num_cats.append(cats)

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.set(xlim=(0,19), ylim=(0, 350))

g = sns.barplot(x=cat_cols, y=num_cats, ax=ax)

for i in range(len(num_cats)):
    g.annotate(str(num_cats[i]), xy=(i, num_cats[i]), xytext=(0,10), textcoords='offset points',
               horizontalalignment='center', fontsize=12)
    
plt.show()

We can can that variable 'cat10' has far more categories than any other variable in this dataset. But we need to make sure that the test dataset doesn't contain more categories than the test datset. Better said, we need to make sure that the test dataset doesn't contain categories that are absent from the training dataset.

In [ ]:
num_cats_test = []

for col in cat_cols:
    cats = test_data[col].nunique()
    num_cats_test.append(cats)

fig, ax = plt.subplots(figsize=(12,6))
ax.set(xlim=(0,19), ylim=(0, 350))

g = sns.barplot(x=cat_cols, y=num_cats_test, ax=ax)

for i in range(len(num_cats_test)):
    g.annotate(str(num_cats_test[i]), xy=(i, num_cats_test[i]), xytext=(0,10), textcoords='offset points',
               horizontalalignment='center', fontsize=12)
    
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))

sns.boxplot(x='variable', y='value', data=pd.melt(train_data[cont_cols]))

plt.show()

In [ ]:
train_copy = train_data.copy().drop(['target'], axis=1)
test_copy = test_data.copy()
all_data = pd.concat([train_copy, test_copy], axis=0)

In [ ]:
label_train = train_data.copy()
label_test = test_data.copy()

label_encoder = LabelEncoder()

for col in cat_cols:
    label_encoder.fit(all_data[col])
    label_train[col] = label_encoder.transform(train_data[col])
    label_test[col] = label_encoder.transform(test_data[col])

In [ ]:
label_train.head(2)

In [ ]:
X_train = label_train.drop(['id', 'target'], axis=1)
y_train = label_train['target']
X_test = label_test.drop(['id'], axis=1)

In [ ]:
# Load model libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

### Logistic Regression

In [ ]:
log_reg = LogisticRegression(random_state=123, max_iter=200, n_jobs=-1)

log_reg.fit(X_train, y_train)

log_preds = log_reg.predict(X_val)

print('Logisitic Regression')
print('Accuracy: ', accuracy_score(y_val, log_preds))
print('ROC_AUC Score: ', roc_auc_score(y_val, log_preds))

### Decision Tree Classifier

In [ ]:
tree_clf = DecisionTreeClassifier(max_depth=7, random_state=123)

tree_clf.fit(X_train, y_train)

tree_preds = tree_clf.predict(X_val)

print('Decision Tree Classifier')
print('Accuracy: ', accuracy_score(y_val, tree_preds))
print('ROC_AUC Score: ', roc_auc_score(y_val, tree_preds))

### XGBoost

In [ ]:
xgb_clf = xgb.XGBClassifier(objective='binary:logistic', n_estimators=500, max_depth=5, learning_rate=0.1,
                            seed=123, eval_metric='auc', tree_method='gpu_hist', use_label_encoder=False)

xgb_clf.fit(X_train, y_train)

xgb_preds = xgb_clf.predict(X_val)

print('XGBoost Classifier')
print('Accuracy: ', accuracy_score(y_val, xgb_preds))
print('ROC_AUC Score: ', roc_auc_score(y_val, xgb_preds))

### Random Forest Classifier

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=400, random_state=123)

rf_clf.fit(X_train, y_train)

rf_preds = rf_clf.predict(X_val)

print('Random Forest Classifier')
print('Accuracy: ', accuracy_score(y_val, rf_preds))
print('ROC_AUC Score: ', roc_auc_score(y_val, rf_preds))

### Ada Boost Classifier

In [ ]:
ada_clf = AdaBoostClassifier(n_estimators=400, learning_rate=0.1, random_state=123)

ada_clf.fit(X_train, y_train)

ada_preds = ada_clf.predict(X_val)

print('Ada Boost Classifier')
print('Accuracy: ', accuracy_score(y_val, ada_preds))
print('ROC_AUC Score: ', roc_auc_score(y_val, ada_preds))

### Combining all into one single ensemble method

In [ ]:
voting_classifier = VotingClassifier(estimators=[('lr', log_reg), ('dt', tree_clf), ('xgbc', xgb_clf), ('rf', rf_clf), ('abc', ada_clf)], voting='soft', n_jobs=-1)

voting_classifier.fit(X_train, y_train)

voting_preds = voting_classifier.predict(X_val)

print('Voting Classifier')
print('Accuracy: ', accuracy_score(y_val, voting_preds))
print('ROC_AUC Score: ', roc_auc_score(y_val, voting_preds))

In [ ]:
voting_probs = voting_classifier.predict_proba(X_test)

In [ ]:
probs = []

for i in range(len(voting_probs)):
    probs.append(voting_probs[i][1])

In [ ]:
submission = pd.DataFrame({'id': label_test['id'], 'target': probs})

submission.to_csv('Submission.csv', index=False)